# 10 Minutes for Streaming from a POP

Welcome to the **10 Minutes for Streaming from a POP** tutorial! This notebook demonstrates the core functionalities of the `scidx_streaming` library, which allows you to create and interact with streaming data using the sciDX platform and a Point of Presence.

In this tutorial, you will learn how to:
1. Set up the library and authenticate with the API.
2. Create, consume, and manage Kafka streams.
3. Use filter semantics for advanced data processing.


## 1. Setting Up

Ensure you have Python 3.7 or higher installed. Use the following steps to install the library and its dependencies:

### Step 1: Install the Library
```bash
!pip install scidx_streaming
```

Or, if you're working in a virtual environment and have cloned the repository, you can install it by running the following command in the root folder:
```bash
!pip install -e .
!pip install -r requirements.txt
```


### Step 2: Enter API Credentials

To start using the library, configure the `APIClient` with your API URL and credentials. You need either:

1. The base URL where the POP API is hosted, and valid username and password for authentication.
2. A valid token for authentication.

#### Option 1: Username and Password Authentication

Use the following code to configure the client using your username and password:

```python
from getpass import getpass

# Prompt the user for API credentials
api_base_url = input("Enter the POP API base URL (include http:// or https://): ")
api_username = input("Enter your POP API username: ")
api_password = getpass("Enter your POP API password: ")
api_token=None
```

#### Option 2: Token-Based Authentication

If you already have a token, you can use it directly instead of providing a username and password:

```python
# Prompt the user for API credentials
api_base_url = input("Enter the POP API base URL (include http:// or https://): ")
api_token = input("Enter your POP API token: ")
api_username=None
api_password=None
```

## 2. Initializing the Client

Now that you have the required credentials, you can import and configure the `pointofpresence` client and the `scidx_streaming` client with it:

```python
from scidx_streaming.client.init_client import StreamingClient
from pointofpresence import APIClient

# Initialize the API client
client = APIClient(base_url=api_base_url, username=api_username, password=api_password, token=api_token)

# Initialize the Streaming client
streaming = StreamingClient(pop_client=client)
print(f"User ID: {streaming.user_id}")
```


## 3. Creating Kafka Streams

You can create Kafka streams with specific keywords and filter semantics. Filter semantics enable advanced processing of streaming data.

### Example: Create a Kafka Stream
```python
# Define the stream configuration
stream_config = {
    "keywords": ["example_keyword", "CSV"],
    "match_all": True,
    "filter_semantics": [
        "window_filter(5, mean, fixed acidity > 8.7)",
        "residual sugar > 1.5",
        "IF window_filter(9, sum, residual sugar > 20) THEN alert = fixed_acidity * 100 ELSE fixed_acidity = fixed_acidity * 1000",
        "IF alert IN ['blue'] THEN residual_sugar = fixed_acidity * 100"
    ]
}

# Create the stream
result = await streaming.create_kafka_stream(
    keywords=stream_config["keywords"],
    filter_semantics=stream_config["filter_semantics"],
    match_all=stream_config["match_all"]
)

print(f"Stream created: {result['topic']}")
print(f"Involved data objects: {result['involved_streams']}")
```



## 4. Consuming Messages from Kafka Streams

Once a stream is created, you can consume messages in real time.

### Example: Consume Kafka Messages
```python
# Consume messages from the Kafka topic
topic = result["topic"]
consumer = streaming.consume_kafka_messages(topic)

try:
    start_time = time.time()
    while True:
        if time.time() - start_time > 30:
            print("Timeout reached while waiting for messages.")
            break

        if not consumer.dataframe.empty:
            print("Dataframe received:")
            print(consumer.dataframe.head())
            break

        await asyncio.sleep(1)
finally:
    consumer.stop()
```



## 4. Deleting Kafka Streams

You can delete a Kafka stream when it is no longer needed.

### Example: Delete a Kafka Stream
```python
# Delete the Kafka stream
response = await streaming.delete_stream(topic)
print(f"Stream {topic} deleted successfully.")
```



## Running Tests

To validate the library's functionalities, you can run the test suite:

```bash
pytest
```



## Conclusion

You have learned how to:
1. Create Kafka streams with custom configurations.
2. Consume messages from Kafka topics.
3. Delete streams when they are no longer needed.

For more details, check the official documentation and [GitHub repository](https://github.com/sci-ndp/streaming-py).
